#### Integrate ChatGPT in your code! ###

https://platform.openai.com/chat-completions

### Use Tavily Search tool ###

https://tavily.com/


In [ ]:
import json
from typing import Dict
from typing_extensions import TypedDict
from dotenv import load_dotenv

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, START, END

In [ ]:
load_dotenv()

In [ ]:
search_query_llm = ChatOpenAI(
    model_name="gpt-4", 
    temperature=0.7,
)

researcher_llm = ChatOpenAI(
    model_name="gpt-4", 
    temperature=0.7,
)

recommend_llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0.7,
)

In [ ]:
# Define the state schema - the keeps track of the conversation history of all agents
class AppState(TypedDict): # Ensures that the state is of type Dict
    messages: list[BaseMessage] # ensures that the messages are of type BaseMessage

In [ ]:
# Initialize Tavily search tool - explain how the agents can use numerous different tools
search = TavilySearchResults(
    max_results=7,
    search_depth="basic",
    exclude_domains=["www.tripadvisor.com", "www.tripadvisor.co.uk", "www.tripadvisor.ca", "www.tripadvisor.in"],
    )

In [ ]:
def search_tool(state: Dict) -> Dict:
    """
    Refines the user's vacation preferences into a specific search query and retrieves search results.

    Args:
        state (Dict): The current state of the conversation, including messages.

    Returns:
        Dict: The updated state with the search results appended to the messages.
    """
    messages = state["messages"]
    
    # First use LLM to refine the search query
    refine_messages = [
        SystemMessage(content="""
            You are a search query specialist.
            
            Your task is to reformulate user vacation preferences into specific, 
            targeted search terms that will yield the most relevant vacation destinations.
                      
            Return only the search query, nothing else."""),

        HumanMessage(content=f"""
            Convert this request into a specific internet search query: 
            {messages[-1].content}""")
    ]

    refined_query = search_query_llm.invoke(refine_messages).content
    
    # Strip quotation marks from refined query
    refined_query = refined_query.replace('"', '')
    print(f"\n==== REFINED QUERY ====\n{refined_query}")
    
    # Use refined query with Tavily for search
    search_results = search.invoke({
        "query": refined_query,
    })
    
    # Format and add results to message history
    formatted_results = json.dumps(search_results, indent=2)
    messages.append(AIMessage(content=formatted_results))
    
    print(f"\n==== SEARCH TOOL RESULTS ====\n{formatted_results}")
    
    return {"messages": messages}

In [ ]:
# Define the research node
def research(state: Dict) -> Dict:
    """
    Analyzes search results and extracts key findings about vacation destinations.
    
    Args:
        state (Dict): The current state of the conversation, including messages.
    
    Returns:
        Dict: The updated state with the research findings appended to the messages.
    """
    messages = state["messages"]
    research_messages = [
        SystemMessage(content='''
            You are an expert vacation planner and research analyst using web content. Analyze the provided search results and extract key vacation 
            destinations along with detailed information about what they offer. 
            
            For each destination, please include:
            - **Destination Name**: The name or location of the vacation spot.
            - **Key Features**: Unique attractions or benefits (e.g., scenic views, cultural sites, adventure activities).
            - **Amenities & Activities**: Information on accommodations, dining, recreational activities, and local experiences.
            - **Pros & Cons**: Brief evaluation points that can help in deciding if the destination fits various user preferences.
            - **Actionable Tips**: Recommendations for planning a visit (e.g., best time to visit, must-see attractions, local travel tips).
            
            Organize your response into clear sections for each destination. Use bullet points or headings where appropriate for clarity.
            Recommend only destinations that are found in the search results. 
        '''),
        *messages # Transfer conversation history
    ]
    response = researcher_llm.invoke(research_messages)
    print(f'\n=====RESEARCH AGENT NODE=====\n{response}')
    return {"messages": messages + [response]}


In [ ]:
# Define the explain node
def recommend(state: Dict) -> Dict:
    """
    Provides professional recommendations for vacation destinations based on research findings.
    
    Args:
        state (Dict): The current state of the conversation, including messages.
    
    Returns:
        Dict: The updated state with the recommendations appended to the messages.
    """
    messages = state["messages"]
    recommendation_messages = [
        SystemMessage(content='''
            You are an expert vacation planner known for providing clear and professional recommendations.
            
            Based on the research findings provided, please perform the following tasks:
            1. Identify and rank the top vacation destinations that best meet the user's query.
            2. For each top destination, provide:
                - **Destination Name**: The vacation spot's name or location.
                - **Information**: A detailed explanation of why this destination is ideal, including unique features, amenities, and any standout attractions.
                - **Recommendations**: Actionable tips or suggestions for planning a visit, such as the best time to travel, local must-see attractions, and any insider advice.
                
            Organize your response in a clear, structured format (using headings or bullet points) to ensure it is easy to understand.
            Be as detailed and informative as necessary, as a travel agent would be when providing a recommendation to a client.
        '''),
        *messages
    ]
    response = recommend_llm.invoke(recommendation_messages)
    print(f"\n=====TRAVEL AGENT NODE====={response}")
    return {"messages": messages + [response]}

In [ ]:
# Build the graph
graph = StateGraph(AppState)
graph.add_node("search", search_tool)
graph.add_node("research", research)
graph.add_node("recommend", recommend)

# Define the edges
graph.set_entry_point("search")
graph.add_edge("search", "research")
graph.add_edge("research", "recommend")
graph.add_edge("recommend", END)

In [ ]:
# Define your function to run the graph
def run_conversation(user_input: str):
    """
    Runs the conversation graph with the given user input.
    
    Args:
        user_input (str): The user's input message.
    
    Returns:
        str: The final output message from the conversation.
    """
    initial_state = {
        "messages": [
            HumanMessage(content=user_input)
        ]
    }
    app = graph.compile()
    output = app.invoke(initial_state)
    return output["messages"][-1].content

result = run_conversation("Im looking for a relaxing spa vacation in a bleak desert environment")
print(result)